<a href="https://colab.research.google.com/github/hvr2026/Autogluon/blob/main/text%20classification/Named_Entity_Recognition_text_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autogluon.multimodal
!pip install torchaudio==2.3.1


In [ ]:
import json
json.dumps([
    {"entity_group": "PERSON", "start": 0, "end": 15},
    {"entity_group": "LOCATION", "start": 28, "end": 35}
])

'[{"entity_group": "PERSON", "start": 0, "end": 15}, {"entity_group": "LOCATION", "start": 28, "end": 35}]'

In [ ]:
from autogluon.multimodal.utils import visualize_ner

sentence = "Albert Einstein was born in Germany and is widely acknowledged to be one of the greatest physicists."
annotation = [{"entity_group": "PERSON", "start": 0, "end": 15},
              {"entity_group": "LOCATION", "start": 28, "end": 35}]

visualize_ner(sentence, annotation)

If you are already familiar with the NER task, you probably have heard about the [BIO](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) (Benginning-Inside-Outside) format. You can adopt this format (which is not compulsory) to add an *I-prefix* or a *B-prefix* to each tag to indicate wether the tag is the beginning of the annotated chunk or inside the chunk. For example, you can annotate "Albert" as "B-PERSON" because it is the beginning of the name and "Einstein" as "I-PERSON" as it is inside the PERSON chunk. You do not need to worry about the *O* tags, an *O* tag indicates that a word belongs to no chunk, as AutoMM will take care of that automatically.

Now, let's look at an example dataset. This dataset is converted from the [MIT movies corpus](https://groups.csail.mit.edu/sls/downloads/movie/) which provides annotations on entity groups such as actor, character, director, genre, song, title, trailer, year, etc.

In [ ]:
from autogluon.core.utils.loaders import load_pd
train_data = load_pd.load('https://automl-mm-bench.s3.amazonaws.com/ner/mit-movies/train_v2.csv')
test_data = load_pd.load('https://automl-mm-bench.s3.amazonaws.com/ner/mit-movies/test_v2.csv')
train_data.head(5)

,text_snippet,entity_annotations
0,what movies star bruce willis,"[{""entity_group"": ""ACTOR"", ""start"": 17, ""end"":..."
1,show me films with drew barrymore from the 1980s,"[{""entity_group"": ""ACTOR"", ""start"": 19, ""end"":..."
2,what movies starred both al pacino and robert ...,"[{""entity_group"": ""ACTOR"", ""start"": 25, ""end"":..."
3,find me all of the movies that starred harold ...,"[{""entity_group"": ""ACTOR"", ""start"": 39, ""end"":..."
4,find me a movie with a quote about baseball in it,[]


Let's print a row.

In [ ]:
print(f"text_snippet: {train_data['text_snippet'][1]}")
print(f"entity_annotations: {train_data['entity_annotations'][1]}")
visualize_ner(train_data['text_snippet'][1], train_data['entity_annotations'][1])

text_snippet: show me films with drew barrymore from the 1980s
entity_annotations: [{"entity_group": "ACTOR", "start": 19, "end": 33}, {"entity_group": "YEAR", "start": 43, "end": 48}]


## Training
Now, let's create a predictor for named entity recognition by setting the *problem_type* to **ner** and specifying the label column. Afterwards, we call predictor.fit() to train the model for five minutes. To achieve reasonable performance in your applications, you are recommended to set a longer enough time_limit (e.g., 30/60 minutes). You can also specify your backbone model and other hyperparameters using the hyperparameters argument. Here, we save the model to the directory `"automm_ner"`. For the purpose of demonstration, we use the lightweighted `'google/electra-small-discriminator'` backbone.

In [ ]:
from autogluon.multimodal import MultiModalPredictor
import uuid

label_col = "entity_annotations"
model_path = f"./tmp/{uuid.uuid4().hex}-automm_ner"  # You can rename it to the model path you like
predictor = MultiModalPredictor(problem_type="ner", label=label_col, path=model_path)
predictor.fit(
    train_data=train_data,
    hyperparameters={'model.ner_text.checkpoint_name':'google/electra-small-discriminator'},
    time_limit=300, #second
)

=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Pytorch Version:    2.3.1+cu121
CUDA Version:       12.1
Memory Avail:       10.95 GB / 12.67 GB (86.4%)
Disk Space Avail:   194.59 GB / 235.68 GB (82.6%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/tmp/3cc02d3db0d94ef185bed02712b67b8c-automm_ner
    ```

INFO: Seed set to 0
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/pyt

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

INFO: Epoch 2, global step 205: 'val_ner_token_f1' reached 0.86904 (best 0.86904), saving model to '/content/tmp/3cc02d3db0d94ef185bed02712b67b8c-automm_ner/epoch=2-step=205.ckpt' as top 3
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Start to fuse 3 checkpoints via the greedy soup algorithm.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/tmp/3cc02d3db0d94ef185bed02712b67b8c-automm_ner")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




## Evaluation
Evaluation is also straightforward, we use [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) for NER evaluation and the supported metrics are *overall_recall*, *overall_precision*, *overall_f1*, *overall_accuracy*. If you are interested in seeing the performance on a specific entity group, you can use the entity group name as the evaluation metric with which you will obtain the performances (precision, recall, f1) on the given entity group:

In [ ]:
predictor.evaluate(test_data,  metrics=['overall_recall', "overall_precision", "overall_f1", "actor"])

Predicting: |          | 0/? [00:00<?, ?it/s]

{'overall_recall': 0.8482861959168384,
 'overall_precision': 0.8323837529865834,
 'overall_f1': 0.8402597402597404,
 'actor': {'precision': 0.8241881298992161,
  'recall': 0.9064039408866995,
  'f1': 0.8633431085043988,
  'number': 812}}

## Prediction + Visualization
You can easily obtain the predictions given an input sentence by by calling predictor.predict(). If you are running the code in a Jupyter notebook, you can also easily visualize the predictions using the `visualize_ner` function which will highlight the named entities and their labels in a text.

In [ ]:
from autogluon.multimodal.utils import visualize_ner

sentence = "Game of Thrones is an American fantasy drama television series created by David Benioff"
predictions = predictor.predict({'text_snippet': [sentence]})
print('Predicted entities:', predictions[0])

# Visualize
visualize_ner(sentence, predictions[0])

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicted entities: [{'entity_group': 'TITLE', 'start': 0, 'end': 15}, {'entity_group': 'GENRE', 'start': 22, 'end': 44}, {'entity_group': 'DIRECTOR', 'start': 74, 'end': 87}]


## Prediction Probabilities

You can also output the probabilities for a deep dive into the predictions.

In [ ]:
predictions = predictor.predict_proba({'text_snippet': [sentence]})
print(predictions[0][0]['probability'])

Predicting: |          | 0/? [00:00<?, ?it/s]

{'O': 0.1539, 'B-GENRE': 0.007965, 'B-TRAILER': 0.000718, 'I-YEAR': 0.0003998, 'I-GENRE': 0.0005035, 'B-YEAR': 0.000626, 'B-RATING': 0.001049, 'B-SONG': 0.02931, 'I-TITLE': 0.009315, 'B-TITLE': 0.7, 'I-DIRECTOR': 0.001594, 'B-REVIEW': 0.00322, 'I-PLOT': 0.002932, 'I-REVIEW': 0.000801, 'I-SONG': 0.002449, 'B-PLOT': 0.04037, 'I-RATING': 0.001203, 'B-RATINGS_AVERAGE': 0.0009665, 'B-DIRECTOR': 0.002237, 'B-CHARACTER': 0.0289, 'I-TRAILER': 8.12e-05, 'B-ACTOR': 0.003944, 'I-RATINGS_AVERAGE': 0.00046, 'I-CHARACTER': 0.004986, 'I-ACTOR': 0.0009627}


## Reloading and Continuous Training
The trained predictor is automatically saved and you can easily reload it using the path. If you are not saftisfied with the current model performance, you can continue training the loaded model with new data.

In [ ]:
new_predictor = MultiModalPredictor.load(model_path)
new_model_path = f"./tmp/{uuid.uuid4().hex}-automm_ner_continue_train"
new_predictor.fit(train_data, time_limit=60, save_path=new_model_path)
test_score = new_predictor.evaluate(test_data, metrics=['overall_f1', 'ACTOR'])
print(test_score)

Load pretrained checkpoint: /content/tmp/3cc02d3db0d94ef185bed02712b67b8c-automm_ner/model.ckpt
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Pytorch Version:    2.3.1+cu121
CUDA Version:       12.1
Memory Avail:       9.24 GB / 12.67 GB (72.9%)
Disk Space Avail:   194.49 GB / 235.68 GB (82.5%)

AutoMM starts to create your model. ✨✨✨

To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/tmp/8b7aa9f5cf1341748ce1dbfb64483316-automm_ner_continue_train
    ```

INFO: Seed set to 0
GPU Count: 1
GPU Count to be Used: 1
GPU 0 Name: Tesla T4
GPU 0 Memory: 0.39GB/15.0GB (Used/Total)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: Tr

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 34: 'val_ner_token_f1' reached 0.86522 (best 0.86522), saving model to '/content/tmp/8b7aa9f5cf1341748ce1dbfb64483316-automm_ner_continue_train/epoch=0-step=34.ckpt' as top 3
INFO: Time limit reached. Elapsed time is 0:01:00. Signaling Trainer to stop.


...

Predicting: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/tmp/8b7aa9f5cf1341748ce1dbfb64483316-automm_ner_continue_train")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




Predicting: |          | 0/? [00:00<?, ?it/s]

{'overall_f1': 0.8380794396829786, 'ACTOR': {'precision': 0.8316268486916951, 'recall': 0.9002463054187192, 'f1': 0.8645771732702542, 'number': 812}}
